#### Packages

In [1]:
import numpy as np
from pandas import DataFrame
import os
import cv2
import shutil
from time import time
from tensorflow.keras.models import load_model

#### Load Images and Apply Model

In [2]:
## load images
path_unseen = input("Please enter the path for the directory that includes unseen images: ")
print("------------------------------------------------------------------------------------------------------------")
start1 = time()
fileList = os.listdir(path_unseen)
x_unseen = []
for file in fileList:
    img = cv2.imread(path_unseen + "//" + file, cv2.IMREAD_GRAYSCALE) # convert to gray scale
    img = cv2.resize(img, (128,128)) # resize unseen images to 128*128, which we worked with in training process
    x_unseen.append(img)
x_unseen = np.array(x_unseen)    
x_unseen = x_unseen / 255 # normalize with the same number (255) that we used in training process
x_unseen = x_unseen.reshape(-1,128,128,1) # target size is 128*128, gray; only 1 channel
end1 = time()
print(f"The execution time to load and process unseen images is {str(end1 - start1)} seconds.")
print("------------------------------------------------------------------------------------------------------------")
## load model and apply it to unseen images
# loaded_model = load_model("model_T.h5") # model_T was learned from training set and was tested on validation set
# loaded_model = load_model("model_TV.h5") # model_TV was learned from training & validation sets and was tested on test set
loaded_model = load_model("model_TVT.h5") # model_TVT was learned from the whole dataset at the end
start2 = time()
pred_t_unseen = loaded_model.predict(x_unseen)
end2 = time()
print(f"The execution time to apply the model to the unseen images is {str(end2 - start2)} seconds.")
print("------------------------------------------------------------------------------------------------------------")
pred_label_unseen = np.argmax(pred_t_unseen, axis = 1)
df = DataFrame({"File Name": fileList, "Predicted Label": pred_label_unseen.tolist()}) # present as a dataframe
display(df)
print("------------------------------------------------------------------------------------------------------------")
df_0 = df[df["Predicted Label"] == 0]
df_1 = df[df["Predicted Label"] == 1]
numPred_0 = df_0.shape[0]
numPred_1 = df_1.shape[0]
print(f"Total number of images with the predicted label of 0 (no background): {numPred_0}")
print(f"Total number of images with the predicted label of 1 (with background): {numPred_1}")
print("------------------------------------------------------------------------------------------------------------")

Please enter the path for the directory that includes unseen images: C:\Users\sinas\OneDrive\Desktop\Test Images\by videos\20220412_140423_frames_test
------------------------------------------------------------------------------------------------------------
The execution time to load and process unseen images is 24.24965190887451 seconds.
------------------------------------------------------------------------------------------------------------
The execution time to apply the model to the unseen images is 3.964491128921509 seconds.
------------------------------------------------------------------------------------------------------------


,File Name,Predicted Label
0,frame1.jpg,1
1,frame10.jpg,1
2,frame100.jpg,1
3,frame1000.jpg,1
4,frame1001.jpg,1
...,...,...
5312,frame995.jpg,1
5313,frame996.jpg,0
5314,frame997.jpg,1
5315,frame998.jpg,1


------------------------------------------------------------------------------------------------------------
Total number of images with the predicted label of 0 (no background): 2341
Total number of images with the predicted label of 1 (with background): 2976
------------------------------------------------------------------------------------------------------------


#### Separate Images of 2 Classes in 2 Subdirectories

In [3]:
## Make two subdirectories (associated with two classes) in which images of each class are copied
start3 = time()
os.mkdir(path_unseen + "//" + "No BG") # class 0
os.mkdir(path_unseen + "//" + "with BG") # class 1
fileNames_0 = list(df_0["File Name"]) # name of images of class 0
fileNames_1 = list(df_1["File Name"]) # name of images of class 1
for file in fileList: # copy process
    if (file in fileNames_0):
        shutil.copy(path_unseen + "//" + file, path_unseen + "//" + "No BG")
    elif (file in fileNames_1):
        shutil.copy(path_unseen + "//" + file, path_unseen + "//" + "with BG")
end3 = time()
print(f"The execution time to separate unseen images into two subdirectories is {str(end3 - start3)} seconds.")
print("------------------------------------------------------------------------------------------------------------")
print("Two subdirectories were made in the path user provided:")
print("1. Images associated with class 0 were copied in the subdirectory 'No BG'")
print("2. Images associated with class 1 were copied in the subdirectory 'with BG'")
print("------------------------------------------------------------------------------------------------------------")

The execution time to separate unseen images into two subdirectories is 9.62803053855896 seconds.
------------------------------------------------------------------------------------------------------------
Two subdirectories were made in the path user provided:
1. Images associated with class 0 were copied in the subdirectory 'No BG'
2. Images associated with class 1 were copied in the subdirectory 'with BG'
------------------------------------------------------------------------------------------------------------
